In [1]:
import pandas as pd
import requests
import json
import glob
import itertools
import re

In [3]:
from sqlalchemy import create_engine

In [4]:
from selenium import webdriver
from bs4 import BeautifulSoup

In [5]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning) 

In [6]:
url = 'https://www.dooinauction.com/auction/search/?km_status=ing&current_page=1&records_per_page=100'
driver = webdriver.Chrome(executable_path='chromedriver.exe')
driver.get(url)
driver.implicitly_wait(30)
soup = BeautifulSoup(driver.page_source, 'lxml')
# driver.close()
last = int(soup.find_all('a','page-link')[-2].text)
print(last)

127


In [7]:
data = pd.DataFrame()

for p in range(last):
    p+=1
    driver = webdriver.Chrome(executable_path='chromedriver.exe')
    url = f'https://www.dooinauction.com/auction/search/?km_status=ing&current_page={p}&records_per_page=100'
    # url = f'https://www.dooinauction.com/auction/search/?km_status=ing&addr1%5B%5D=%EC%84%9C%EC%9A%B8&addr1%5B%5D=%EA%B2%BD%EA%B8%B0&current_page={p}&records_per_page=100'
    driver.get(url)
    print(url)
    driver.implicitly_wait(30)
    soup = BeautifulSoup(driver.page_source, 'lxml')
    
    items = soup.find_all("div","list-product")
    # if len(items) ==0:
    #     print("BREAK")
    #     break
        
    # driver.close()
    df = pd.DataFrame()
    for i in items:
        d = pd.DataFrame()
        one = i.find_all('span',{'address-txt'})
        d['경매번호'] = [one[0].text.replace('지도 보기 ','')]
        d['주소지'] = [one[1].text]
        try:
            d['x'] = [(str(one[0]).split('urlX=')[1].split('&')[0])]
            d['y'] = [(str(one[0]).split('urlY=')[1].split('"')[0])]
        except IndexError:
            d['x'] = [""]
            d['y'] = [""]
        except ValueError:
            d['x'] = [""]
            d['y'] = [""]
        two = i.find_all('div',{'court-txt'})
        d['종류']  = [two[0].text]
        try:
            three = i.find_all(lambda tag: tag.name == 'div' and tag.get('class') == ['size-txt'])
            d['면적'] = [three[0].text]
        except IndexError:
            d['면적'] = ['']
        
        four = i.find_all('div','price-value')
        try:
            d['감정가'] = [int(four[0].text.replace(',',""))]
            d['입찰가'] = [int(four[1].text.replace(',',""))]
        except IndexError:
            d['감정가'] = [0]
            d['입찰가'] = [0]
        
        five = i.find_all('div', 'date-day')        
        d['입찰일'] = [five[0].text]
        
        six = i.find_all('input',value=True)[0]['value']
        try:
            d['링크'] = [six]
        except IndexError:
            d['링크'] = [0]
        df = df.append(d)
        
    data = data.append(df)
    

https://www.dooinauction.com/auction/search/?km_status=ing&current_page=1&records_per_page=100
https://www.dooinauction.com/auction/search/?km_status=ing&current_page=2&records_per_page=100
https://www.dooinauction.com/auction/search/?km_status=ing&current_page=3&records_per_page=100
https://www.dooinauction.com/auction/search/?km_status=ing&current_page=4&records_per_page=100
https://www.dooinauction.com/auction/search/?km_status=ing&current_page=5&records_per_page=100
https://www.dooinauction.com/auction/search/?km_status=ing&current_page=6&records_per_page=100
https://www.dooinauction.com/auction/search/?km_status=ing&current_page=7&records_per_page=100
https://www.dooinauction.com/auction/search/?km_status=ing&current_page=8&records_per_page=100
https://www.dooinauction.com/auction/search/?km_status=ing&current_page=9&records_per_page=100
https://www.dooinauction.com/auction/search/?km_status=ing&current_page=10&records_per_page=100
https://www.dooinauction.com/auction/search/?km_s

In [13]:
# data['감정가'] = data['감정가'].str.replace(',','')

In [8]:
data['크기'] = 10

data['x'] = pd.to_numeric(data['x'],errors='coerce')
data['y'] = pd.to_numeric(data['y'],errors='coerce')

# data['감정가']=data['감정가'].apply(lambda x : "{:,}".format(x))
# data['입찰가']=data['입찰가'].apply(lambda x : "{:,}".format(x))

data['감정가'] = pd.to_numeric(data['감정가'],errors='coerce')
data['입찰가'] = pd.to_numeric(data['입찰가'],errors='coerce')

data['감정가']=data['감정가'].apply("{:,}".format)
data['입찰가']=data['입찰가'].apply("{:,}".format)

data['링크'] = data["링크"].apply(lambda x: '<a href="https://www.dooinauction.com/auction/search/?auction_id={0}">link</a>'.format(x))

In [9]:
data['건물 면적'] = data['면적'].astype(str).str.split('/').apply(lambda x: [s for s in x if '건물' in s]).str[0].str.split('(').str[1].str.extract(r'(\d+.\d+)').astype('float')

data['토지 면적'] = data['면적'].astype(str).str.split('/').apply(lambda x: [s for s in x if '토지' in s]).str[0].str.split('(').str[1].str.extract(r'(\d+.\d+)').astype('float')

data['대지권 면적'] = data['면적'].astype(str).str.split('/').apply(lambda x: [s for s in x if '대지권' in s]).str[0].str.split('(').str[1].str.extract(r'(\d+.\d+)').astype('float')

In [10]:
data.fillna(0,inplace=True)

In [11]:
data.to_excel('data.xls',index=False)

data.to_csv("data.csv",encoding ='euc_kr',index=False)

In [58]:
import plotly as py
import plotly.graph_objs as go
import plotly.express as px
py.offline.init_notebook_mode(connected = True)

In [60]:
data.head()

,경매번호,주소지,x,y,종류,면적,감정가,입찰가,입찰일,링크,크기,건물 면적,토지 면적,대지권 면적
0,2016 타경 2590(1),제주 제주시 구좌읍 송당리 15-1 외 1개 목록,126.810426,33.458084,잡종지,토지 7497㎡(2267.84평),"337,365,000","337,365,000",2022-12-20,"<a href=""https://www.dooinauction.com/auction/...",10,0.00,2267.84,0.0
1,2016 타경 2590(2),제주 제주시 구좌읍 송당리 20,126.809556,33.456340,임야,토지 3101㎡(938.05평),"155,050,000","155,050,000",2022-12-20,"<a href=""https://www.dooinauction.com/auction/...",10,0.00,938.05,0.0
2,2016 타경 2590(3),제주 제주시 구좌읍 송당리 22,126.808829,33.454936,잡종지,토지 3537㎡(1069.94평),"180,387,000","180,387,000",2022-12-20,"<a href=""https://www.dooinauction.com/auction/...",10,0.00,1069.94,0.0
3,2019 타경 1303(1),강원 평창군 대화면 하안미리 1440-1 외 7개 목록,128.451146,37.479753,농가시설,건물 2787.27㎡(843.15평) / 토지 5527㎡(1671.92평) / 제시...,"866,712,890","866,712,890",2022-12-20,"<a href=""https://www.dooinauction.com/auction/...",10,843.15,1671.92,0.0
4,2019 타경 1303(2),강원 평창군 대화면 대화리 630 외 1개 목록,128.467469,37.468682,전,토지 14942㎡(4519.96평),"281,188,000","281,188,000",2022-12-20,"<a href=""https://www.dooinauction.com/auction/...",10,0.00,4519.96,0.0


In [64]:
df

,경매번호,주소지,x,y,종류,면적,감정가,입찰가,입찰일,링크,크기,건물 면적,토지 면적,대지권 면적


In [69]:
df = data.copy()
df = df[df['주소지'].str.contains('|'.join([]))]
df = df[df['건물 면적']>=100]
df = df[df['토지 면적']>=0]

px.set_mapbox_access_token("pk.eyJ1IjoidGl2bWU3IiwiYSI6ImNrMWEwZDVtNDI4Zm4zYm1vY3o3Z25zejEifQ._yTPkj3nXTzor72zIevLCQ")
hover = [x for x in ['종류','크기','x','y','링크','경매번호','주소지','건물 면적','토지 면적','대지권 면적','감정가','입찰가','입찰일'] if x in data.columns]
fig = px.scatter_mapbox(df, lat="y", lon="x", hover_name="주소지",zoom = 6, hover_data = hover,color='종류',size = '크기')
fig.update_traces(visible='legendonly', selector=dict(type='scattermapbox'))
# fig.update_layout(autosize=True,width=1500,height=750)
# plot = (py.offline.plot(fig,output_type="div"))
py.offline.plot(fig)

'temp-plot.html'